In [1]:
import pandas as pd
import glob
import datetime
import re
import warnings
import numpy as np
warnings.filterwarnings(action='ignore')

In [2]:
file_list = glob.glob('./data/*.xlsx')
# file_list

In [3]:
# 정류장 번호
place = {
    '경주골굴사':['02089','02090'], '경주김유신묘':['01025','01036'], 
    '경주동궁과월지':['01237', '01238'], '경주동부사적지대':['01279','01280'], 
    '경주무열왕릉':['01194','01195'], '경주문무대왕릉':['02056','02057'],
    '경주불국사':['01160','01161'], '경주석굴암':['01199','01199'], 
    '경주신문왕릉':['01081','01082'], '경주월성(반월성)':['01237','01238'], 
    '경주첨성대':['01279','01280'], '경주금곡사':['05379','05380'],
    '경주기림사':['02021','02021'], '경주백률사':['01228','01229'],
    '경주분황사':['01157','01158'],'경주연지암':['05423','05426'], 
    '경주주사암':['03121','03122']
} 

In [4]:
bus_final = {'일반10': ['20', '21:40'],
             '일반11': ['20', '22:00'],
             '일반15': ['300', '18:30'],
             '일반16': ['100', '19:30'],
             '일반231': ['100', '19:20'],
             '일반234': ['100', '17:20'],
             '일반273': ['90', '20:00'],
             '일반30': ['80', '20:27'],
             '일반301': ['30', '19:15'],
             '일반31': ['80', '20:10'],
             '일반333': ['120', '18:25'],
             '일반334': ['85', '20:45'],
             '일반335': ['130', '18:40'],
             '일반337': ['120', '20:00'],
             '일반338': ['100', '19:25'],
             '일반40': ['27', '21:55'],
             '일반41': ['30', '21:55'],
             '일반50': ['20', '22:55'],
             '일반602': ['320', '18:30'],
             '일반604': ['155', '17:45'],
             '일반605': ['80', '20:00'],
             '일반607': ['100', '18:00'],
             '좌석10': ['20', '21:40'],
             '좌석100': ['20', '22:00'],
             '좌석100-1': ['20', '22:00'],
             '좌석11': ['20', '22:00'],
             '좌석12': ['60', '18:20'],
             '좌석130': ['60', '19:15'],
             '좌석150': ['50', '21:40'],
             '좌석150-1': ['50', '17:40'],
             '좌석18': ['115', '20:20'],
             '좌석300': ['30', '21:25'],
             '좌석303': ['170', '21:00'],
             '좌석332': ['100', '20:05'],
             '좌석350': ['30', '21:45'],
             '좌석601': ['105', '19:25'],
             '좌석700': ['60', '18:20'],
             '좌석710': ['60', '20:46'],
             '좌석711': ['60', '20:23']}
bus_final_list = list(bus_final.items())
# bus_final_list

In [5]:
df_list = []
for i in file_list:
    temp = pd.read_excel(i)
    del temp['Unnamed: 0']
    df_list.append(temp)
# df_list[0]

In [6]:
for i in range(len(bus_final_list)):
    for j in range(len(df_list)):
        if bus_final_list[i][0] == df_list[j]['버스번호'][0]:
            df_list[j]['배차간격'] = bus_final_list[i][1][0]
            df_list[j]['막차'] = bus_final_list[i][1][1]
# df_list[0]

In [7]:
# 시간 변환 함수

def time_convert(time):
    hour = 0
    minute = 0
    second = 0
    if time >= 60:
        hour = time // 60
        minute = time % 60
    else:
        minute = time
    return [hour, minute]

In [8]:
##### 데이터 전처리 #####

### 하나의 리스트에 모든 버스 데이터프레임 저장
file_list = glob.glob('./data/*.xlsx')
df_list = []

for i in file_list:
    temp = pd.read_excel(i)
    del temp['Unnamed: 0']
    df_list.append(temp)
    
### 리스트 내 전체 데이터 전처리
for x in range(len(df_list)):
    
    # 도착시간 ':' 제거
    for i in range(1, len(df_list[x])):
        if df_list[x]['도착시간'][i][0] == '0':
            df_list[x]['도착시간'][i] = '2분'
    
    # 도착시간에 시간 더해가기
    time = int(df_list[x]['도착시간'][1].split('분')[0])
    sum_time = 0
    for i in range(1, len(df_list[x]['도착시간'])):
        time = int(df_list[x]['도착시간'][i].split('분')[0])
        sum_time += time
        df_list[x]['도착시간'].loc[i] = sum_time      

In [14]:
### '첫차', '막차', '배차'에 따른 데이터 전처리
for x in range(len(df_list)):
    # '배차간격', '막차' 컬럼 추가
    for i in range(len(bus_final_list)):
        if bus_final_list[i][0] == df_list[x]['버스번호'][0]:
            df_list[x]['배차간격'] = bus_final_list[i][1][0]
            df_list[x]['막차'] = bus_final_list[i][1][1]

In [57]:
# 배차 컬럼 100개 생성
name = [f'배차{i}' for i in range(1, 101)]
for x in range(len(df_list)):
    for i in range(100):
        df_list[x][name[i]] = df_list[x]['도착시간']

# 배차 별 배차간격 시간 더하기
# for x in range(len(df_list)):
        term = int(df_list[x]['배차간격'][0])
        temp = 0
        for i in range(100):
            hour = int(df_list[x][name[i]][0].split(':')[0])
            minute = int(df_list[x][name[i]][0].split(':')[1])

            temp += minute
            cal = temp + term
            df_list[x][name[i]][0] = str(hour) + ':' + str(cal)
            b = time_convert(cal)
            df_list[x][name[i]][0] = str(hour+b[0]) + ':' + str(b[1])   
            # 시간 변환하기
            if cal >= 60:
                b = time_convert(cal)
                df_list[x][name[i]][0] = str(hour+b[0]) + ':' + str(b[1])

                    # 막차 시간 이후 컬럼은 삭제
                end_hour = int(df_list[x]['막차'][0].split(':')[0])
                end_minute = int(df_list[x]['막차'][0].split(':')[1])

                if (end_hour <= hour+b[0] and end_minute <= b[1]) or (end_hour < hour+b[0]):
                    del df_list[x][name[i]]

KeyError: '배차47'

In [61]:
df_list[3]

,버스노선,정류장번호,버스번호,도착시간,배차간격,막차,배차1,배차2,배차3,배차4,...,배차29,배차30,배차31,배차32,배차33,배차34,배차35,배차36,배차37,배차38
0,시외버스.고속버스터미널,1225,일반15,07:10,300,18:30,12:10,12:20,12:30,12:40,...,16:50,17:0,17:10,17:20,17:30,17:40,17:50,18:0,18:10,18:20
1,고속버스.시외버스터미널,1045,일반15,1,300,18:30,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,서라벌사거리.황리단길,1190,일반15,4,300,18:30,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
3,중앙시장,1309,일반15,5,300,18:30,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,법원.동산병원 건너,1083,일반15,6,300,18:30,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
5,신한은행사거리,1232,일반15,8,300,18:30,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
6,경주우체국,1031,일반15,9,300,18:30,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
7,경고지하도,1007,일반15,10,300,18:30,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
8,팔우정,1351,일반15,11,300,18:30,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
9,월성동주민센타,1280,일반15,12,300,18:30,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12


In [13]:
a = df_list[30][df_list[30].columns[6:]]
a.iloc[:, 0]

0     8:35
1        1
2        2
3        3
4        4
5        5
6        7
7        8
8        9
9       10
10      11
11      12
12      13
13      14
14      15
15      16
16      17
17      18
18      20
19      21
20      22
21      23
22      24
23      25
24      26
25      27
26      28
27      29
28      30
29      31
30      32
31      33
32      34
33      36
34      37
35      38
36      39
37      40
38      41
39      42
40      43
41      44
42      45
43      46
44      47
45      48
46      49
47      50
48      51
49      52
Name: 배차1, dtype: object

In [14]:
df_list[0]['도착시간'][1]

1

In [15]:
# 배차 컬럼별 정류장 시간 계산
for x in range(len(df_list)):
    for i in range(1, len(df_list[x])): 
        if df_list[x]['도착시간'][i] >= 60:
            b = time_convert(df_list[x]['도착시간'][i])
            df_list[x]['도착시간'][i] = str(int(df_list[x]['도착시간'][0].split(':')[0]) + b[0]) + ':' + str(int(df_list[x]['도착시간'][0].split(':')[1]) + b[1])


        df_temp = df_list[x][df_list[x].columns[6:]]
#         a.iloc[:, 0]
        for j in range(1, len(df_temp.columns)):
            if int(df_temp.iloc[:, j-1][0].split(":")[1]) >= 60:
                c = time_convert(int(df_temp.iloc[:, j-1][i].split(':')[1]))
                df_temp.iloc[:, j-1][i] = str(int(df_temp.iloc[:, j-1][0].split(':')[0]) + c[0]) + ':' + str(int(df_temp.iloc[:, j-1][0].split(':')[1]) + c[1])
            


In [16]:
df_list[30]

,버스노선,정류장번호,버스번호,도착시간,배차간격,막차,배차1,배차2,배차3,배차4,...,배차64,배차65,배차66,배차67,배차68,배차69,배차70,배차71,배차72,배차73
0,성동시장,1030,일반334,07:10,85,20:45,8:35,8:45,8:55,9:5,...,19:5,19:15,19:25,19:35,19:45,19:55,20:5,20:15,20:25,20:35
1,신한은행사거리,1233,일반334,1,85,20:45,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,법원.동산병원,1137,일반334,2,85,20:45,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,중앙시장,1308,일반334,3,85,20:45,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,서라벌사거리.황리단길,1189,일반334,4,85,20:45,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
5,고속버스터미널,1044,일반334,5,85,20:45,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
6,경주여중.원자력환경공단.코라드,1025,일반334,7,85,20:45,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
7,경주초등학교앞,1037,일반334,8,85,20:45,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
8,서라벌대학.신라고,1185,일반334,9,85,20:45,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
9,서라벌대학기숙사앞.송화도서관,1188,일반334,10,85,20:45,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10


In [17]:
##### 데이터 입출력
# 장소 입력

# 시간 입력

# 장소와 시간에 따른 정류장과 시간표 출력

In [18]:
# tmp = 0
# baecha_time = int(df_list[2]['배차간격'][0])

# df_list[2][baecha_name[0]] = df_list[2]['도착시간']
# tmp = baecha_time
# time_test = time_convert(tmp)
# df_list[2][baecha_name[0]][0] = str(int(df_list[2][baecha_name[0]][0].split(':')[0]) + time_test[0]) \
#     + ':' + str(int(df_list[2][baecha_name[0]][0].split(':')[1]) + time_test[1])

# df_list[2][baecha_name[1]] = df_list[2]['도착시간']
# tmp += baecha_time
# time_test = time_convert(tmp)
# df_list[2][baecha_name[1]][0] = str(int(df_list[2][baecha_name[1]][0].split(':')[0]) + time_test[0]) \
#     + ':' + str(int(df_list[2][baecha_name[1]][0].split(':')[1]) + time_test[1])


# df_list[2][baecha_name[2]] = df_list[2]['도착시간']
# tmp += baecha_time
# time_test = time_convert(tmp)
# df_list[2][baecha_name[2]][0] = str(int(df_list[2][baecha_name[2]][0].split(':')[0]) + time_test[0]) \
#     + ':' + str(int(df_list[2][baecha_name[2]][0].split(':')[1]) + time_test[1])




# df_list[2]
# # + int(df_list[2]['배차간격'][0])
# # df_list[2]    